# APLICACIÓN

En este archivo se encuentra la aplicación con los dos modelos desplegados:
- Transfer-learning Model: Es la arquitectura que tiene la transferencia de capas desde un modelo pre-entrenado (lo pre-entrenamos nosotros mismos -archivo subBigGAN.ipynb-) con los datos de CIFAR10.


- Scratch Model: Este modelo fue entrenado por la arquitectura de cGAN desde 0 con el total de imágenes.

In [54]:
import gradio as gr
import os
from tensorflow import train
#from helpers import load_data # la función fue creada por nostros, para más info ver el notebook de data.ipynb
import matplotlib.pyplot as plt
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from tensorflow.keras.models import Model, load_model # for assembling a Neural Network model
from tensorflow.keras.layers import Input, Dense, Embedding, Reshape, Concatenate, Flatten, Dropout # for adding layers
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, ReLU, LeakyReLU # for adding layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model # for plotting model diagram
from tensorflow.keras.optimizers import Adam # for model optimization 
import numpy as np
import h5py

Tensorflow/Keras: 2.11.0


## LOGIC



In [31]:
def scratch_generator(latent_dim, in_shape=(4,4,1), n_cats=3):
    # TODO: cambiar inshape a 8x8x1
    # Label Inputs
    in_label = Input(shape=(1,), name='Generator-Label-Input-Layer') # Input Layer
    lbls = Embedding(input_dim = n_cats, output_dim = 50,
                     name='Generator-Label-Embedding-Layer')(in_label) # Embed label to vector
    
    # Generator Inputs (latent vector)
    in_latent = Input(shape=latent_dim, name='Generator-Latent-Input-Layer')
    # Scale up to image dimensions
    n_nodes = in_shape[0] * in_shape[1] 
    lbls = Dense(n_nodes, name='Generator-Label-Dense-Layer')(lbls)
    lbls = Reshape((in_shape[0], in_shape[1], 1), name='Generator-Label-Reshape-Layer')(lbls) # New shape

    # Generator Inputs (latent vector)
    in_latent = Input(shape=latent_dim, name='Generator-Latent-Input-Layer')
    
    # Image Foundation 
    n_nodes = in_shape[0] * in_shape[1] * 256 
    g = Dense(n_nodes, name='Generator-Foundation-Layer')(in_latent)
    g = ReLU(name='Generator-Foundation-Layer-Activation-1')(g)
    g = Reshape((in_shape[0], in_shape[1], 256), name='Generator-Foundation-Layer-Reshape-1')(g)
    
    # Combine both inputs so it has two channels
    concat = Concatenate(name='Generator-Combine-Layer')([g, lbls])

    # Hidden Layer 1
    g = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='Generator-Hidden-Layer-1')(concat)
    g = LeakyReLU(alpha=0.2, name='Generator-Hidden-Layer-Activation-1')(g)
    
    # Hidden Layer 2
    g = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='Generator-Hidden-Layer-2')(g)
    g = LeakyReLU(alpha=0.2, name='Generator-Hidden-Layer-Activation-2')(g)
    
    # Hidden Layer 3
    g = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='Generator-Hidden-Layer-3')(g)
    g = LeakyReLU(alpha=0.2, name='Generator-Hidden-Layer-Activation-3')(g)
    
    # Hidden Layer 4
    # g = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='Generator-Hidden-Layer-4')(g)
    # g = ReLU(name='Generator-Hidden-Layer-Activation-4')(g)
    
    # Output Layer (Note, we use only one filter because we have a greysclae image. Color image would have three
    output_layer = Conv2D(filters=3, kernel_size=(7,7), activation='tanh', padding='same', name='Generator-Output-Layer')(g)
    # TODO: cambiar filters a 3
    # Define model
    model = Model([in_latent, in_label], output_layer, name='Generator')
    return model

def transfer_generator(latent_dim, in_shape=(4,4,1), n_cats=3):
    # TODO: cambiar inshape a 8x8x1
    # Label Inputs
    in_label = Input(shape=(1,), name='Generator-Label-Input-Layer') # Input Layer
    lbls = Embedding(input_dim = n_cats, output_dim = 50,
                     name='Generator-Label-Embedding-Layer')(in_label) # Embed label to vector
    
    # Generator Inputs (latent vector)
    in_latent = Input(shape=latent_dim, name='Generator-Latent-Input-Layer')
    # Scale up to image dimensions
    n_nodes = in_shape[0] * in_shape[1] 
    lbls = Dense(n_nodes, name='Generator-Label-Dense-Layer')(lbls)
    lbls = Reshape((in_shape[0], in_shape[1], 1), name='Generator-Label-Reshape-Layer')(lbls) # New shape

    # Generator Inputs (latent vector)
    in_latent = Input(shape=latent_dim, name='Generator-Latent-Input-Layer')
    
    # Image Foundation 
    n_nodes = in_shape[0] * in_shape[1] * 256 
    g = Dense(n_nodes, name='Generator-Foundation-Layer')(in_latent)
    g = ReLU(name='Generator-Foundation-Layer-Activation-1')(g)
    g = Reshape((in_shape[0], in_shape[1], 256), name='Generator-Foundation-Layer-Reshape-1')(g)
    
    # Combine both inputs so it has two channels
    concat = Concatenate(name='Generator-Combine-Layer')([g, lbls])

    # Hidden Layer 1
    g = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='Generator-Hidden-Layer-1', trainable=False)(concat)
    g = LeakyReLU(alpha=0.2, name='Generator-Hidden-Layer-Activation-1', trainable=False)(g)
    
    # Hidden Layer 2
    g = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='Generator-Hidden-Layer-2', trainable=False)(g)
    g = LeakyReLU(alpha=0.2, name='Generator-Hidden-Layer-Activation-2', trainable=False)(g)
    
    # Hidden Layer 3
    g = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='Generator-Hidden-Layer-3', trainable=False)(g)
    g = LeakyReLU(alpha=0.2, name='Generator-Hidden-Layer-Activation-3', trainable=False)(g)
    
    # Hidden Layer 4
    # g = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding='same', name='Generator-Hidden-Layer-4')(g)
    # g = ReLU(name='Generator-Hidden-Layer-Activation-4')(g)
    g = Conv2DTranspose(filters=128, kernel_size=(4,4), padding='same', name='Generator-Hidden-Layer-4')(g)
    g = LeakyReLU(alpha=0.2, name='Generator-Hidden-Layer-Activation-4')(g)
    
    # Hidden Layer 5
    g = Conv2DTranspose(filters=128, kernel_size=(4,4), padding='same', name='Generator-Hidden-Layer-5')(g)
    g = LeakyReLU(alpha=0.2, name='Generator-Hidden-Layer-Activation-5')(g)
    
    # Output Layer (Note, we use only one filter because we have a greysclae image. Color image would have three
    output_layer = Conv2D(filters=3, kernel_size=(7,7), activation='tanh', padding='same', name='Generator-Output-Layer')(g)
    # TODO: cambiar filters a 3
    # Define model
    model = Model([in_latent, in_label], output_layer, name='Generator')
    return model


In [40]:
class Generator:
    def __init__(self, checkpoint_dir_scratch, checkpoint_dir_transfer):
        #instantiate generators
        self.scratch = scratch_generator(latent_dim=100)
        self.transfer = transfer_generator(latent_dim=100)
        # get latest checkpoint
        latest_scratch = train.latest_checkpoint(checkpoint_dir_scratch)
        latest_transfer = train.latest_checkpoint(checkpoint_dir_transfer)
        # load weights
        self.scratch.load_weights(latest_scratch)
        self.transfer.load_weights(latest_transfer)
        self.labels_map = {"Abstract": 0, "Geometric": 1, "Islamic": 2}
    
    def generate(self, label: str, type_: str):
        label = self.labels_map[label]
        latent_points, _ = self.__latent_vector(100, 1)
        if type_ == "scratch":
            img  = self.scratch.predict([latent_points, np.array([label])])
        elif type_ == "transfer":
            img  = self.transfer.predict([latent_points, np.array([label])])
        img = (img+1)/2.0
        return img[0]
    
    def __latent_vector(self, latent_dim, n, n_cats=3):
        # Generate points in the latent space
        latent_input = np.random.randn(latent_dim * n)

        # Reshape into a batch of inputs for the network
        latent_input = latent_input.reshape(n, latent_dim)

        # Generate category labels 
        cat_labels = np.random.randint(0, n_cats, n)
        return [latent_input, cat_labels] 

In [41]:
checkpoint_dir_scratch = "checkpoints/checkpoints_scratch32"
latest_scratch = train.latest_checkpoint(checkpoint_dir_scratch)
checkpoint_dir_transfer = "checkpoints/checkpoints_transfer32"
latest_transfer = train.latest_checkpoint(checkpoint_dir_transfer)

GAN = Generator(checkpoint_dir_scratch, checkpoint_dir_transfer)

# APP DEPLOYMENT

In [67]:
def generate(Type_Art, Model_type):
    label = Type_Art.split(" ")[0]
    type_ = "transfer" if Model_type =="Transfer-learning Model" else "scratch"
    return GAN.generate(label, type_)

demo = gr.Interface(
    fn=generate,
    inputs=[gr.Radio(["Abstract Art", "Geometric Art", "Islamic Art"]), gr.Radio(["Transfer-learning Model", "Scratch Model"])],
    outputs=gr.Image(shape=(32, 32)), allow_flagging="never"
)
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7877
Running on public URL: https://6e8eb49350d8db91.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 75ms/step
